In [ ]:
# Cell 1: Setup and Configuration
# AgriConnect - Agricultural Intelligence Multi-Agent System
# Capstone Project for AI Agent Development Workshop

import os
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional
import json

# Mount Google Drive for datasets
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install -q google-genai pillow pandas

# Import Google AI libraries
from google import genai
from google.genai import types
from PIL import Image

# Import ADK libraries
from google.adk.agents import Agent
from google.adk.models import Gemini
from google.adk.tools import FunctionTool, google_search
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

print("✅ All packages imported successfully")
print("=" * 70)

In [ ]:
# Cell 2: API Configuration and Client Setup

# Get API key from Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Validate API key
if not GOOGLE_API_KEY or len(GOOGLE_API_KEY) < 20:
    raise ValueError("❌ Invalid API key! Please add GOOGLE_API_KEY to Colab secrets")

print(f"✅ API Key loaded: {GOOGLE_API_KEY[:10]}...{GOOGLE_API_KEY[-4:]}")
print(f"   Length: {len(GOOGLE_API_KEY)} characters")

# Initialize Gemini client
gemini_client = genai.Client(api_key=GOOGLE_API_KEY)

# Find best available Gemini model
print("\n🔍 Finding best Gemini model...")
try:
    models = gemini_client.models.list()

    # Filter for text generation Gemini models (exclude imagen, embedding, etc.)
    gemini_models = [
        m.name for m in models
        if 'gemini' in m.name.lower()
        and 'imagen' not in m.name.lower()
        and 'embedding' not in m.name.lower()
        and 'live' not in m.name.lower()
        and 'tts' not in m.name.lower()
        and 'robotics' not in m.name.lower()
        and 'computer-use' not in m.name.lower()
    ]

    print(f"✅ Found {len(gemini_models)} suitable Gemini models")

    # Prefer these models in order (for text/vision generation)
    preferred_models = [
        'models/gemini-2.5-flash',
        'models/gemini-2.0-flash-exp',
        'models/gemini-2.0-flash',
        'models/gemini-flash-latest',
    ]

    GEMINI_MODEL = None
    for model in preferred_models:
        if model in gemini_models:
            GEMINI_MODEL = model
            break


    if not GEMINI_MODEL and gemini_models:
        GEMINI_MODEL = gemini_models[0]

    if not GEMINI_MODEL:
        raise ValueError("No suitable Gemini model found")

    print(f"✅ Selected model: {GEMINI_MODEL}")

except Exception as e:
    print(f"❌ Failed to select model: {e}")
    GEMINI_MODEL = "models/gemini-2.5-flash"
    print(f"⚠️  Using fallback: {GEMINI_MODEL}")

# Test API connection
print("\n🔌 Testing API connection...")
try:
    test_response = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents="Say 'Connected!' in one word."
    )
    print(f"✅ API Test successful: {test_response.text.strip()}")
    print("✅ API is working correctly!")
except Exception as e:
    print(f"❌ API Test Failed: {e}")
    print("   Please check your API key and quota")

print("\n" + "=" * 70)
print("✅ Configuration complete")
print(f"   Primary model: {GEMINI_MODEL}")
print("=" * 70)

In [ ]:
# Cell 3: Dataset Configuration

# Define dataset paths in Google Drive
BASE_PATH = Path('/content/drive/MyDrive/AgriConnect_data')
MAIZE_PATH = BASE_PATH / 'corn_or_maize'
RICE_PATH = BASE_PATH / 'rice-leaf-desease'

print("📁 Dataset Configuration")
print("=" * 70)
print(f"Base Path: {BASE_PATH}")
print(f"Maize Path: {MAIZE_PATH}")
print(f"Rice Path: {RICE_PATH}")

# Verify paths exist
if BASE_PATH.exists():
    print("\n✅ Base path exists")

    if MAIZE_PATH.exists():
        maize_classes = [d.name for d in MAIZE_PATH.iterdir() if d.is_dir()]
        print(f"✅ Maize dataset found: {len(maize_classes)} classes")
        print(f"   Classes: {', '.join(maize_classes)}")
    else:
        print("⚠️ Maize path not found")

    if RICE_PATH.exists():
        rice_classes = [d.name for d in RICE_PATH.iterdir() if d.is_dir()]
        print(f"✅ Rice dataset found: {len(rice_classes)} classes")
        print(f"   Classes: {', '.join(rice_classes)}")
    else:
        print("⚠️ Rice path not found")
else:
    print("❌ Base path does not exist!")
    print("   Please check your Google Drive structure")

print("=" * 70)

In [ ]:
# Cell 4: Comprehensive Disease Knowledge Base
# This serves as a custom tool for detailed disease information

DISEASE_DATABASE = {
    "maize": {
        "blight": {
            "display_name": "Northern Leaf Blight",
            "scientific_name": "Exserohilum turcicum",
            "severity": "High",
            "yield_loss": "15-30% (can reach 70% in severe cases)",
            "symptoms": [
                "Long, elliptical gray-green lesions on leaves",
                "Lesions run parallel to leaf veins",
                "Lesions turn tan or brown as they mature",
                "Can cover entire leaf in severe infections",
                "Lower leaves affected first, moving upward"
            ],
            "treatment": [
                "Apply fungicides: Mancozeb, Propiconazole, or Azoxystrobin",
                "Begin application at first sign of disease",
                "Repeat applications every 7-14 days",
                "Ensure good spray coverage on both leaf surfaces",
                "Remove and destroy heavily infected plants"
            ],
            "prevention": [
                "Plant resistant or tolerant maize hybrids",
                "Practice crop rotation (2-3 year cycle)",
                "Plow under crop residue after harvest",
                "Maintain proper plant spacing for air circulation",
                "Avoid overhead irrigation",
                "Apply balanced fertilization"
            ],
            "local_name_hausa": "Cutar Ganyen Masara ta Arewa",
            "special_notes": "Most damaging in areas with high humidity and moderate temperatures (18-27°C)"
        },
        "common_rust": {
            "display_name": "Common Rust",
            "scientific_name": "Puccinia sorghi",
            "severity": "Moderate to High",
            "yield_loss": "30-50% in susceptible varieties",
            "symptoms": [
                "Circular to elongate brown pustules on leaves",
                "Pustules appear on both upper and lower leaf surfaces",
                "Reddish-brown powdery spores (rust) when rubbed",
                "Pustules may merge causing large dead areas",
                "Severely infected leaves die prematurely"
            ],
            "treatment": [
                "Apply fungicides: Azoxystrobin or Mancozeb",
                "Spray when first symptoms appear",
                "Repeat application if disease persists",
                "Time applications based on disease pressure",
                "Early treatment is most effective"
            ],
            "prevention": [
                "Plant resistant maize varieties",
                "Ensure proper plant spacing",
                "Apply adequate nitrogen but avoid excess",
                "Early planting can help avoid peak disease periods",
                "Monitor fields regularly during humid weather"
            ],
            "local_name_hausa": "Tsatsa (Common Rust)",
            "special_notes": "Favored by cool nights (16-23°C) and warm days with high humidity"
        },
        "gray_leaf_spot": {
            "display_name": "Gray Leaf Spot",
            "scientific_name": "Cercospora zeae-maydis",
            "severity": "Very High",
            "yield_loss": "Up to 65% in severe cases",
            "symptoms": [
                "Small, rectangular brown lesions between leaf veins",
                "Lesions are parallel-sided (boxcar shape)",
                "Gray appearance when fungal spores develop",
                "Lesions may merge to kill entire leaves",
                "Lower leaves affected first"
            ],
            "treatment": [
                "Apply strobilurin or triazole fungicides",
                "Treat at first signs of disease",
                "Multiple applications may be needed",
                "Fungicides most effective when applied early",
                "Consider combination products for best control"
            ],
            "prevention": [
                "Plant resistant or tolerant varieties",
                "Use crop rotation",
                "Bury crop residue through tillage",
                "Ensure balanced nutrition",
                "Avoid water stress"
            ],
            "local_name_hausa": "Tabo Mai Launin Toka",
            "special_notes": "Most severe under warm (25-30°C), humid conditions with frequent rainfall"
        },
        "healthy": {
            "display_name": "Healthy Maize",
            "symptoms": [
                "Uniform dark green color across all leaves",
                "No visible spots, lesions, or discoloration",
                "Vigorous growth with strong stems",
                "Normal leaf structure and texture"
            ],
            "advice": [
                "Continue regular monitoring",
                "Maintain current care practices",
                "Scout fields weekly for early disease detection",
                "Ensure adequate water and nutrients",
                "Keep records of healthy plant appearance"
            ],
            "local_name_hausa": "Masara Mai Koshin Lafiya"
        }
    },
    "rice": {
        "bacterial_leaf_blight": {
            "display_name": "Bacterial Leaf Blight",
            "scientific_name": "Xanthomonas oryzae pv. oryzae",
            "severity": "Very High",
            "yield_loss": "20-80% depending on stage and severity",
            "symptoms": [
                "Water-soaked lesions on leaf tips and margins",
                "Lesions turn yellow then white/gray as they dry",
                "Wavy margins on lesions (diagnostic feature)",
                "Yellow bacterial ooze on leaves in morning",
                "Severe infection causes whole leaf death",
                "Can affect grain filling"
            ],
            "treatment": [
                "NO CURATIVE TREATMENT - prevention is critical",
                "Copper-based bactericides may slow spread",
                "Remove and burn infected plants",
                "Avoid working in fields when wet",
                "Sanitize all farming tools between uses"
            ],
            "prevention": [
                "MOST IMPORTANT: Plant resistant varieties",
                "Use certified disease-free seed",
                "Avoid excessive nitrogen fertilization",
                "Maintain proper water management",
                "Remove crop residues after harvest",
                "Practice crop rotation where possible",
                "Avoid injuries to plants during field operations"
            ],
            "local_name_hausa": "Cutar Ganyen Shinkafa ta Kwayoyin Cuta",
            "special_notes": "NO CURE exists. Bacteria spread rapidly in warm, humid conditions. Prevention through resistant varieties is ESSENTIAL."
        },
        "brown_spot": {
            "display_name": "Brown Spot",
            "scientific_name": "Bipolaris oryzae",
            "severity": "Moderate",
            "yield_loss": "10-50% in severe cases",
            "symptoms": [
                "Small, circular to oval brown spots on leaves",
                "Spots have dark brown border with gray center",
                "Numerous spots can coalesce",
                "Also affects grain, reducing quality",
                "Associated with nutrient deficiency"
            ],
            "treatment": [
                "Apply fungicides: Mancozeb or Copper oxychloride",
                "Spray at tillering and flowering stages",
                "Repeat if disease pressure is high",
                "Address nutrient deficiencies"
            ],
            "prevention": [
                "Use disease-free, high-quality seed",
                "Apply balanced fertilization (especially N and K)",
                "Maintain proper water management",
                "Remove alternate host plants (weeds)",
                "Plant resistant varieties if available"
            ],
            "local_name_hausa": "Tabo Mai Launin Ruwan Kasa",
            "special_notes": "Often indicates poor soil fertility. Correct nutrition often reduces disease severity."
        },
        "leaf_smut": {
            "display_name": "Leaf Smut",
            "scientific_name": "Entyloma oryzae",
            "severity": "Low to Moderate",
            "yield_loss": "5-20% in most cases",
            "symptoms": [
                "Small, angular, black spots on both leaf surfaces",
                "Spots are slightly raised",
                "Velvety appearance due to fungal spores",
                "Spots may be surrounded by yellow halo",
                "Usually doesn't kill entire leaves"
            ],
            "treatment": [
                "Fungicides: Mancozeb or Copper oxychloride",
                "Usually not economically justified unless severe",
                "Focus on prevention rather than treatment"
            ],
            "prevention": [
                "Use resistant varieties when available",
                "Maintain proper water management",
                "Avoid excessive plant density",
                "Ensure good field drainage",
                "Apply balanced nutrition"
            ],
            "local_name_hausa": "Bakin Cuta akan Ganyen Shinkafa",
            "special_notes": "Generally causes minor damage. Resistant varieties provide best control."
        },
        "healthy": {
            "display_name": "Healthy Rice",
            "symptoms": [
                "Uniform green leaves throughout plant",
                "No spots, lesions, or discoloration",
                "Normal growth and development",
                "Strong, upright plant structure"
            ],
            "advice": [
                "Continue monitoring regularly",
                "Maintain proper water levels",
                "Ensure adequate but not excessive fertilization",
                "Scout for pests and diseases weekly"
            ],
            "local_name_hausa": "Shinkafa Mai Koshin Lafiya"
        }
    }
}

# Helper function to retrieve disease information
def get_disease_info(crop_type: str, disease_key: str) -> Dict[str, Any]:
    """
    Retrieve detailed disease information from knowledge base

    Args:
        crop_type: 'maize' or 'rice'
        disease_key: disease identifier (e.g., 'blight', 'common_rust')

    Returns:
        Dictionary with disease details
    """
    crop_data = DISEASE_DATABASE.get(crop_type.lower(), {})
    return crop_data.get(disease_key.lower(), {})

def list_all_diseases(crop_type: str) -> List[str]:
    """List all diseases for a crop type"""
    crop_data = DISEASE_DATABASE.get(crop_type.lower(), {})
    return [info.get('display_name', key) for key, info in crop_data.items()]

print("✅ Disease Knowledge Base loaded")
print("=" * 70)
print(f"Maize conditions: {len(DISEASE_DATABASE['maize'])}")
print(f"Rice conditions: {len(DISEASE_DATABASE['rice'])}")
print("\nAvailable diseases:")
print(f"  Maize: {', '.join(list_all_diseases('maize'))}")
print(f"  Rice: {', '.join(list_all_diseases('rice'))}")
print("=" * 70)

In [ ]:
# Cell 5: API Quota Management System


class QuotaManager:
    """
    Manages API quota to prevent hitting rate limits
    Tracks requests per minute and enforces delays
    """

    def __init__(self, max_requests_per_minute=5):
        self.max_rpm = max_requests_per_minute
        self.request_times = []
        self.total_requests = 0
        self.by_agent = {}

    def can_make_request(self) -> bool:
        """Check if we can make a request without hitting rate limit"""
        current_time = time.time()

        # Remove requests older than 60 seconds
        self.request_times = [t for t in self.request_times if current_time - t < 60]

        return len(self.request_times) < self.max_rpm

    def wait_if_needed(self, agent_name: str = "Unknown"):
        """Wait if necessary to stay within rate limits"""
        if not self.can_make_request():
            # Calculate wait time
            oldest_request = min(self.request_times)
            wait_time = 60 - (time.time() - oldest_request) + 1

            if wait_time > 0:
                print(f"   ⏳ [{agent_name}] Rate limit: waiting {int(wait_time)}s for quota reset...")
                time.sleep(wait_time)
                self.request_times = []

    def record_request(self, agent_name: str = "Unknown"):
        """Record that we made a request"""
        self.request_times.append(time.time())
        self.total_requests += 1

        # Track by agent
        if agent_name not in self.by_agent:
            self.by_agent[agent_name] = 0
        self.by_agent[agent_name] += 1

    def get_stats(self) -> Dict[str, Any]:
        """Get usage statistics"""
        return {
            'total_requests': self.total_requests,
            'requests_last_minute': len(self.request_times),
            'quota_available': self.max_rpm - len(self.request_times),
            'by_agent': self.by_agent
        }

    def print_stats(self):
        """Print current quota status"""
        stats = self.get_stats()
        print(f"\n📊 API Quota Status:")
        print(f"   Total requests: {stats['total_requests']}")
        print(f"   Last minute: {stats['requests_last_minute']}/{self.max_rpm}")
        print(f"   Available: {stats['quota_available']}")
        if stats['by_agent']:
            print(f"   By agent:")
            for agent, count in stats['by_agent'].items():
                print(f"      • {agent}: {count}")

# Initialize global quota manager
quota_manager = QuotaManager(max_requests_per_minute=5)

print("✅ Quota Manager initialized")
print(f"   Rate limit: {quota_manager.max_rpm} requests/minute")
print("=" * 70)

In [ ]:
# Cell 6: Crop Pathologist Agent
# Uses Gemini Vision API to analyze crop images and diagnose diseases

import base64
from typing import Dict, Tuple

class CropPathologistAgent:
    """
    Specialist agent for diagnosing crop diseases from images
    Uses Gemini Vision API + Disease Knowledge Base
    """

    def __init__(self, gemini_client, model_name: str, quota_manager):
        self.client = gemini_client
        self.model_name = model_name
        self.quota_manager = quota_manager
        self.agent_name = "Crop Pathologist"
        self.diagnoses_count = 0

    def analyze_image(self, image_path: str, crop_type: str) -> Dict[str, any]:
        """
        Analyze crop image for disease diagnosis

        Args:
            image_path: Path to crop leaf image
            crop_type: 'maize' or 'rice'

        Returns:
            Dictionary with diagnosis results
        """

        print(f"\n🔬 [{self.agent_name}] Starting analysis...")
        print(f"   Crop type: {crop_type.upper()}")
        print(f"   Image: {Path(image_path).name}")

        start_time = time.time()

        try:
            # Load image
            image = Image.open(image_path)
            print(f"   ✓ Image loaded: {image.size[0]}x{image.size[1]} pixels")

            # Get available diseases for this crop
            crop_diseases = DISEASE_DATABASE.get(crop_type.lower(), {})
            disease_names = [info.get('display_name', key)
                           for key, info in crop_diseases.items()]

            print(f"   ✓ Checking against {len(disease_names)} conditions")

            # Create diagnosis prompt
            prompt = f"""You are an expert agricultural pathologist analyzing a {crop_type} crop leaf image.

AVAILABLE CONDITIONS FOR {crop_type.upper()}:
{', '.join(disease_names)}

YOUR TASK:
1. Carefully examine the leaf image for any disease symptoms
2. Identify which condition from the list best matches what you see
3. Assess the severity and confidence of your diagnosis
4. Describe the key visual features that led to your conclusion

IMPORTANT GUIDELINES:
- Only diagnose conditions from the list above
- If the leaf appears healthy, identify it as "Healthy {crop_type.title()}"
- Be specific about visual symptoms (color, pattern, location, size)
- Consider disease stage (early, moderate, severe)

REQUIRED OUTPUT FORMAT:
Condition: [Exact name from the list above]
Confidence: [High/Medium/Low]
Severity: [None/Mild/Moderate/Severe]
Disease Stage: [N/A or Early/Moderate/Advanced]
Key Visual Features:
- [Feature 1]
- [Feature 2]
- [Feature 3]

Provide your expert diagnosis now."""

            # Wait for quota availability
            self.quota_manager.wait_if_needed(self.agent_name)

            # Make API call to Gemini Vision
            print(f"   🤖 Calling Gemini Vision API...")

            response = self.client.models.generate_content(
                model=self.model_name,
                contents=[image, prompt]
            )

            # Record API usage
            self.quota_manager.record_request(self.agent_name)
            self.diagnoses_count += 1

            elapsed_time = time.time() - start_time
            print(f"   ✓ Analysis complete in {elapsed_time:.2f}s")

            # Parse response
            diagnosis_text = response.text
            diagnosis_data = self._parse_diagnosis(diagnosis_text, crop_type)

            # Enrich with knowledge base information
            diagnosis_data = self._enrich_with_knowledge_base(
                diagnosis_data, crop_type
            )

            # Add metadata
            diagnosis_data['metadata'] = {
                'agent': self.agent_name,
                'model': self.model_name,
                'analysis_time': elapsed_time,
                'image_path': str(image_path),
                'image_size': f"{image.size[0]}x{image.size[1]}",
                'timestamp': datetime.now().isoformat()
            }

            print(f"   ✅ Diagnosis: {diagnosis_data['condition']}")
            print(f"   ✅ Confidence: {diagnosis_data['confidence']}")

            return diagnosis_data

        except Exception as e:
            error_time = time.time() - start_time
            print(f"   ❌ Error after {error_time:.2f}s: {str(e)[:100]}")

            return {
                'condition': 'Analysis Failed',
                'confidence': 'N/A',
                'severity': 'Unknown',
                'error': str(e),
                'raw_output': '',
                'metadata': {
                    'agent': self.agent_name,
                    'error_time': error_time,
                    'timestamp': datetime.now().isoformat()
                }
            }

    def _parse_diagnosis(self, diagnosis_text: str, crop_type: str) -> Dict[str, any]:
        """Parse the diagnosis text from Gemini into structured data"""

        lines = diagnosis_text.strip().split('\n')

        diagnosis_data = {
            'condition': 'Unknown',
            'confidence': 'Low',
            'severity': 'Unknown',
            'disease_stage': 'N/A',
            'key_features': [],
            'raw_output': diagnosis_text
        }

        features_section = False

        for line in lines:
            line = line.strip()

            if line.startswith('Condition:'):
                diagnosis_data['condition'] = line.replace('Condition:', '').strip()

            elif line.startswith('Confidence:'):
                diagnosis_data['confidence'] = line.replace('Confidence:', '').strip()

            elif line.startswith('Severity:'):
                diagnosis_data['severity'] = line.replace('Severity:', '').strip()

            elif line.startswith('Disease Stage:'):
                diagnosis_data['disease_stage'] = line.replace('Disease Stage:', '').strip()

            elif 'Key Visual Features:' in line or 'Visual Features:' in line:
                features_section = True

            elif features_section and line.startswith('-'):
                feature = line.lstrip('- ').strip()
                if feature:
                    diagnosis_data['key_features'].append(feature)

        return diagnosis_data

    def _enrich_with_knowledge_base(self, diagnosis_data: Dict, crop_type: str) -> Dict:
        """Add detailed information from disease knowledge base"""

        condition_name = diagnosis_data['condition']

        # Try to find matching disease in knowledge base
        crop_diseases = DISEASE_DATABASE.get(crop_type.lower(), {})

        matched_disease = None
        matched_key = None

        # Try exact match first
        for key, info in crop_diseases.items():
            if info.get('display_name', '').lower() == condition_name.lower():
                matched_disease = info
                matched_key = key
                break

        # Try partial match if exact match fails
        if not matched_disease:
            for key, info in crop_diseases.items():
                if condition_name.lower() in info.get('display_name', '').lower():
                    matched_disease = info
                    matched_key = key
                    break

        # Add knowledge base information
        if matched_disease:
            diagnosis_data['disease_key'] = matched_key
            diagnosis_data['scientific_name'] = matched_disease.get('scientific_name', 'N/A')
            diagnosis_data['yield_loss'] = matched_disease.get('yield_loss', 'Unknown')
            diagnosis_data['symptoms'] = matched_disease.get('symptoms', [])
            diagnosis_data['treatment'] = matched_disease.get('treatment', matched_disease.get('advice', []))
            diagnosis_data['prevention'] = matched_disease.get('prevention', [])
            diagnosis_data['local_name_hausa'] = matched_disease.get('local_name_hausa', '')
            diagnosis_data['special_notes'] = matched_disease.get('special_notes', '')
            diagnosis_data['knowledge_base_match'] = True
        else:
            diagnosis_data['knowledge_base_match'] = False
            print(f"   ⚠️  No knowledge base match found for: {condition_name}")

        return diagnosis_data

    def format_diagnosis_report(self, diagnosis_data: Dict) -> str:
        """Format diagnosis into a readable report"""

        report = f"""
{'=' * 70}
🔬 CROP PATHOLOGIST DIAGNOSIS REPORT
{'=' * 70}

📋 DIAGNOSIS:
   Condition: {diagnosis_data['condition']}
   Confidence: {diagnosis_data['confidence']}
   Severity: {diagnosis_data['severity']}
   Disease Stage: {diagnosis_data['disease_stage']}
"""

        if diagnosis_data.get('scientific_name'):
            report += f"   Scientific Name: {diagnosis_data['scientific_name']}\n"

        if diagnosis_data.get('local_name_hausa'):
            report += f"   Hausa Name: {diagnosis_data['local_name_hausa']}\n"

        if diagnosis_data.get('yield_loss'):
            report += f"\n📉 POTENTIAL YIELD LOSS: {diagnosis_data['yield_loss']}\n"

        if diagnosis_data.get('key_features'):
            report += f"\n🔍 KEY VISUAL FEATURES OBSERVED:\n"
            for feature in diagnosis_data['key_features']:
                report += f"   • {feature}\n"

        if diagnosis_data.get('symptoms'):
            report += f"\n⚠️ TYPICAL SYMPTOMS OF THIS DISEASE:\n"
            for symptom in diagnosis_data['symptoms'][:3]:  # Show top 3
                report += f"   • {symptom}\n"

        if diagnosis_data.get('treatment'):
            report += f"\n💊 TREATMENT RECOMMENDATIONS:\n"
            for treatment in diagnosis_data['treatment']:
                report += f"   • {treatment}\n"

        if diagnosis_data.get('prevention'):
            report += f"\n🛡️ PREVENTION STRATEGIES:\n"
            for prevention in diagnosis_data['prevention'][:3]:  # Show top 3
                report += f"   • {prevention}\n"

        if diagnosis_data.get('special_notes'):
            report += f"\n📌 SPECIAL NOTES:\n   {diagnosis_data['special_notes']}\n"

        report += f"\n{'=' * 70}\n"

        return report

    def get_statistics(self) -> Dict[str, any]:
        """Get agent statistics"""
        return {
            'agent_name': self.agent_name,
            'total_diagnoses': self.diagnoses_count,
            'api_calls_made': self.quota_manager.by_agent.get(self.agent_name, 0)
        }

# Initialize Crop Pathologist Agent
crop_pathologist = CropPathologistAgent(
    gemini_client=gemini_client,
    model_name=GEMINI_MODEL,
    quota_manager=quota_manager
)

print("✅ Crop Pathologist Agent initialized")
print(f"   Using model: {GEMINI_MODEL}")
print(f"   Ready to analyze crop images")
print("=" * 70)

In [ ]:
# Cell 8: Quality Assessor Agent
# Analyzes overall crop health, vigor, and quality from the same image

class QualityAssessorAgent:
    """
    Specialist agent for assessing overall crop health and quality
    Complements disease diagnosis with holistic plant health evaluation
    """

    def __init__(self, gemini_client, model_name: str, quota_manager):
        self.client = gemini_client
        self.model_name = model_name
        self.quota_manager = quota_manager
        self.agent_name = "Quality Assessor"
        self.assessments_count = 0

    def assess_health(self, image_path: str, crop_type: str,
                      disease_info: Dict = None) -> Dict[str, any]:
        """
        Assess overall crop health and quality from image

        Args:
            image_path: Path to crop leaf image
            crop_type: 'maize' or 'rice'
            disease_info: Optional disease diagnosis info for context

        Returns:
            Dictionary with health assessment results
        """

        print(f"\n📊 [{self.agent_name}] Starting health assessment...")
        print(f"   Crop type: {crop_type.upper()}")

        start_time = time.time()

        try:
            # Load image
            image = Image.open(image_path)
            print(f"   ✓ Image loaded")

            # Create assessment prompt
            prompt = f"""You are an expert agricultural specialist assessing the overall health and quality of a {crop_type} plant from a leaf image.

YOUR TASK:
Conduct a comprehensive health assessment covering these aspects:

1. OVERALL HEALTH SCORE (0-100)
   - Rate the overall plant vigor and health
   - Consider: color, structure, turgor, damage extent

2. LEAF COLOR ASSESSMENT
   - Evaluate the green color intensity and uniformity
   - Identify any discoloration (yellowing, browning, bleaching)
   - Assess if color indicates nutrient status

3. LEAF STRUCTURE & VIGOR
   - Evaluate leaf firmness and turgor
   - Check for wilting or drooping
   - Assess leaf texture and thickness

4. GROWTH STAGE (if determinable)
   - Early vegetative / Mid vegetative / Late vegetative / Reproductive

5. NUTRIENT STATUS
   - Signs of nitrogen deficiency (yellowing)
   - Signs of other nutrient deficiencies
   - Overall nutrient balance

6. DAMAGE ASSESSMENT
   - Extent of visible damage (None/Minimal/Moderate/Severe)
   - Percentage of leaf area affected
   - Impact on photosynthetic capacity

7. HARVEST/MANAGEMENT READINESS
   - Is the plant at optimal health for its stage?
   - Recommended actions

REQUIRED OUTPUT FORMAT:
Health Score: [0-100]
Overall Rating: [Excellent/Good/Fair/Poor/Critical]
Leaf Color: [Description and rating]
Leaf Vigor: [Strong/Moderate/Weak]
Growth Stage: [Stage]
Nutrient Status: [Assessment]
Damage Extent: [Percentage and description]
Photosynthetic Capacity: [High/Medium/Low/Severely Reduced]
Key Observations:
- [Observation 1]
- [Observation 2]
- [Observation 3]
Recommended Actions:
- [Action 1]
- [Action 2]

Provide your expert assessment now."""

            # Wait for quota
            self.quota_manager.wait_if_needed(self.agent_name)

            # Make API call
            print(f"   🤖 Calling Gemini Vision API...")

            response = self.client.models.generate_content(
                model=self.model_name,
                contents=[image, prompt]
            )

            # Record API usage
            self.quota_manager.record_request(self.agent_name)
            self.assessments_count += 1

            elapsed_time = time.time() - start_time
            print(f"   ✓ Assessment complete in {elapsed_time:.2f}s")

            # Parse response
            assessment_text = response.text
            assessment_data = self._parse_assessment(assessment_text)

            # Add context from disease diagnosis if available
            if disease_info:
                assessment_data['disease_context'] = {
                    'condition': disease_info.get('condition', 'Unknown'),
                    'severity': disease_info.get('severity', 'Unknown')
                }

            # Add metadata
            assessment_data['metadata'] = {
                'agent': self.agent_name,
                'model': self.model_name,
                'analysis_time': elapsed_time,
                'timestamp': datetime.now().isoformat()
            }

            print(f"   ✅ Health Score: {assessment_data['health_score']}/100")
            print(f"   ✅ Rating: {assessment_data['overall_rating']}")

            return assessment_data

        except Exception as e:
            error_time = time.time() - start_time
            print(f"   ❌ Error after {error_time:.2f}s: {str(e)[:100]}")

            return {
                'health_score': 0,
                'overall_rating': 'Assessment Failed',
                'error': str(e),
                'metadata': {
                    'agent': self.agent_name,
                    'error_time': error_time
                }
            }

    def _parse_assessment(self, assessment_text: str) -> Dict[str, any]:
        """Parse the assessment text into structured data"""

        lines = assessment_text.strip().split('\n')

        assessment_data = {
            'health_score': 0,
            'overall_rating': 'Unknown',
            'leaf_color': '',
            'leaf_vigor': '',
            'growth_stage': '',
            'nutrient_status': '',
            'damage_extent': '',
            'photosynthetic_capacity': '',
            'key_observations': [],
            'recommended_actions': [],
            'raw_output': assessment_text
        }

        observations_section = False
        actions_section = False

        for line in lines:
            line = line.strip()

            if line.startswith('Health Score:'):
                score_str = line.replace('Health Score:', '').strip()
                # Extract number
                import re
                numbers = re.findall(r'\d+', score_str)
                if numbers:
                    assessment_data['health_score'] = int(numbers[0])

            elif line.startswith('Overall Rating:'):
                assessment_data['overall_rating'] = line.replace('Overall Rating:', '').strip()

            elif line.startswith('Leaf Color:'):
                assessment_data['leaf_color'] = line.replace('Leaf Color:', '').strip()

            elif line.startswith('Leaf Vigor:'):
                assessment_data['leaf_vigor'] = line.replace('Leaf Vigor:', '').strip()

            elif line.startswith('Growth Stage:'):
                assessment_data['growth_stage'] = line.replace('Growth Stage:', '').strip()

            elif line.startswith('Nutrient Status:'):
                assessment_data['nutrient_status'] = line.replace('Nutrient Status:', '').strip()

            elif line.startswith('Damage Extent:'):
                assessment_data['damage_extent'] = line.replace('Damage Extent:', '').strip()

            elif line.startswith('Photosynthetic Capacity:'):
                assessment_data['photosynthetic_capacity'] = line.replace('Photosynthetic Capacity:', '').strip()

            elif 'Key Observations:' in line:
                observations_section = True
                actions_section = False

            elif 'Recommended Actions:' in line:
                actions_section = True
                observations_section = False

            elif observations_section and line.startswith('-'):
                obs = line.lstrip('- ').strip()
                if obs:
                    assessment_data['key_observations'].append(obs)

            elif actions_section and line.startswith('-'):
                action = line.lstrip('- ').strip()
                if action:
                    assessment_data['recommended_actions'].append(action)

        return assessment_data

    def format_assessment_report(self, assessment_data: Dict) -> str:
        """Format assessment into a readable report"""

        # Determine health status color/emoji
        score = assessment_data['health_score']
        if score >= 80:
            status_emoji = "🟢"
            status_text = "EXCELLENT"
        elif score >= 60:
            status_emoji = "🟡"
            status_text = "GOOD"
        elif score >= 40:
            status_emoji = "🟠"
            status_text = "FAIR - NEEDS ATTENTION"
        else:
            status_emoji = "🔴"
            status_text = "POOR - URGENT ACTION NEEDED"

        report = f"""
{'=' * 70}
📊 QUALITY ASSESSOR REPORT
{'=' * 70}

{status_emoji} OVERALL HEALTH ASSESSMENT:
   Health Score: {assessment_data['health_score']}/100
   Rating: {assessment_data['overall_rating']}
   Status: {status_text}

🌿 DETAILED ANALYSIS:
   Leaf Color: {assessment_data['leaf_color']}
   Leaf Vigor: {assessment_data['leaf_vigor']}
   Growth Stage: {assessment_data['growth_stage']}
   Photosynthetic Capacity: {assessment_data['photosynthetic_capacity']}

🧪 NUTRIENT STATUS:
   {assessment_data['nutrient_status']}

⚠️ DAMAGE ASSESSMENT:
   {assessment_data['damage_extent']}
"""

        if assessment_data.get('disease_context'):
            report += f"\n🔬 DISEASE CONTEXT:\n"
            report += f"   Diagnosed Condition: {assessment_data['disease_context']['condition']}\n"
            report += f"   Severity: {assessment_data['disease_context']['severity']}\n"

        if assessment_data.get('key_observations'):
            report += f"\n🔍 KEY OBSERVATIONS:\n"
            for obs in assessment_data['key_observations']:
                report += f"   • {obs}\n"

        if assessment_data.get('recommended_actions'):
            report += f"\n📋 RECOMMENDED ACTIONS:\n"
            for action in assessment_data['recommended_actions']:
                report += f"   • {action}\n"

        report += f"\n{'=' * 70}\n"

        return report

    def get_statistics(self) -> Dict[str, any]:
        """Get agent statistics"""
        return {
            'agent_name': self.agent_name,
            'total_assessments': self.assessments_count,
            'api_calls_made': self.quota_manager.by_agent.get(self.agent_name, 0)
        }

# Initialize Quality Assessor Agent
quality_assessor = QualityAssessorAgent(
    gemini_client=gemini_client,
    model_name=GEMINI_MODEL,
    quota_manager=quota_manager
)

print("✅ Quality Assessor Agent initialized")
print(f"   Using model: {GEMINI_MODEL}")
print(f"   Ready to assess crop health")
print("=" * 70)

In [ ]:
# Cell 10: Geo-Context Agent with Flexible Market Queries

class GeoContextAgent:
    """
    Specialist agent for location-based context:
    - Weather forecasts for specific location
    - Market prices for user-specified crop(s)
    - Regional disease alerts
    """

    def __init__(self, quota_manager):
        self.quota_manager = quota_manager
        self.agent_name = "Geo-Context"
        self.analyses_count = 0

        # Nigerian state capitals and major agricultural centers
        self.locations = {
            'kano': 'Kano, Nigeria',
            'kaduna': 'Kaduna, Nigeria',
            'sokoto': 'Sokoto, Nigeria',
            'katsina': 'Katsina, Nigeria',
            'borno': 'Maiduguri, Borno, Nigeria',
            'yobe': 'Damaturu, Yobe, Nigeria',
            'lagos': 'Lagos, Nigeria'
        }

        # Supported crops for market intelligence
        self.supported_crops = ['maize', 'rice', 'sorghum', 'millet', 'groundnut']

    def get_context(self,
                   location: str,
                   weather_crop: str = None,
                   market_crops: List[str] = None,
                   disease_info: Dict = None) -> Dict[str, any]:
        """
        Get geographical context with flexible crop queries

        USER INPUTS:
        - location: Where the farmer is located (required)
        - weather_crop: Crop for weather-specific advice (optional)
        - market_crops: List of crops to get prices for (USER SPECIFIED)
        - disease_info: Disease diagnosis for weather relevance (optional)

        Returns:
            Dictionary with weather and market context
        """

        print(f"\n🌍 [{self.agent_name}] Gathering location context...")
        print(f"   Location: {location}")

        if market_crops:
            print(f"   Market prices requested for: {', '.join(market_crops).upper()}")
        if weather_crop:
            print(f"   Weather advisory for: {weather_crop.upper()}")

        start_time = time.time()

        # Normalize location
        location_lower = location.lower()
        full_location = self.locations.get(location_lower, location)

        context_data = {
            'location': location,
            'weather': {},
            'market': {},
            'farming_advisory': []
        }

        try:
            # Get weather context
            print(f"   🌤️ Getting weather data for {full_location}...")
            weather_data = self._get_weather_context(
                full_location,
                weather_crop,
                disease_info
            )
            context_data['weather'] = weather_data

            # Get market prices for requested crops
            if market_crops:
                print(f"   💰 Getting market prices...")
                market_data = self._get_market_context_multi(location, market_crops)
                context_data['market'] = market_data
            else:
                context_data['market'] = {
                    'message': 'No crops specified for market intelligence'
                }

            # Generate farming advisory
            context_data['farming_advisory'] = self._generate_farming_advisory(
                weather_data,
                context_data['market'],
                disease_info
            )

            elapsed_time = time.time() - start_time
            context_data['metadata'] = {
                'agent': self.agent_name,
                'analysis_time': elapsed_time,
                'timestamp': datetime.now().isoformat()
            }

            self.analyses_count += 1

            print(f"   ✓ Context gathered in {elapsed_time:.2f}s")

            return context_data

        except Exception as e:
            error_time = time.time() - start_time
            print(f"   ❌ Error after {error_time:.2f}s: {str(e)[:100]}")

            return {
                'location': location,
                'error': str(e),
                'metadata': {
                    'agent': self.agent_name,
                    'error_time': error_time
                }
            }

    def _get_weather_context(self, location: str, crop: str = None,
                            disease_info: Dict = None) -> Dict:
        """Get weather data for location with optional crop-specific advice"""

        # Northern Nigeria typical weather (November - dry season)
        weather_data = {
            'summary': 'Warm and dry conditions',
            'temperature': '28-35°C',
            'temperature_range': {'min': 28, 'max': 35},
            'humidity': '30-50%',
            'humidity_range': {'min': 30, 'max': 50},
            'rainfall': 'Low probability (5-10%)',
            'rainfall_amount': '0-5mm expected',
            'wind': 'Light breeze, 10-15 km/h',
            'conditions': 'Clear to partly cloudy',
            'forecast_period': '7 days',
            'data_source': 'Regional forecast for Northern Nigeria',
            'disease_relevance': [],
            'crop_specific_advice': []
        }

        # Add crop-specific weather advice
        if crop:
            if crop.lower() == 'maize':
                weather_data['crop_specific_advice'].append(
                    "✓ Dry conditions suitable for maize harvest"
                )
                weather_data['crop_specific_advice'].append(
                    "⚠️ Ensure adequate irrigation if planting now"
                )
            elif crop.lower() == 'rice':
                weather_data['crop_specific_advice'].append(
                    "⚠️ Rice requires consistent water - irrigation essential in dry season"
                )
                weather_data['crop_specific_advice'].append(
                    "✓ Low disease pressure in dry conditions"
                )

        # Add disease-specific weather relevance
        if disease_info:
            condition = disease_info.get('condition', '').lower()

            if 'rust' in condition:
                weather_data['disease_relevance'].append(
                    "⚠️ Dry conditions can slow rust spread, but disease is already established"
                )
                weather_data['disease_relevance'].append(
                    "✓ Low humidity is favorable for controlling rust progression"
                )
            elif 'blight' in condition:
                weather_data['disease_relevance'].append(
                    "✓ Dry weather helps limit blight spread"
                )
                weather_data['disease_relevance'].append(
                    "⚠️ Monitor for any humidity increases that could worsen disease"
                )

        return weather_data

    def _get_market_context_multi(self, location: str,
                                  crop_list: List[str]) -> Dict:
        """
        Get market prices for multiple crops (USER SPECIFIED)

        Args:
            location: Farmer's location
            crop_list: List of crops user wants prices for
        """

        market_data = {
            'location': location,
            'crops_queried': crop_list,
            'prices_by_crop': {},
            'comparison': {}
        }

        # Get prices for each requested crop
        for crop in crop_list:
            crop_lower = crop.lower()

            if crop_lower not in self.supported_crops:
                market_data['prices_by_crop'][crop] = {
                    'error': f'Price data not available for {crop}'
                }
                continue

            crop_prices = self._get_single_crop_prices(crop_lower)
            market_data['prices_by_crop'][crop] = crop_prices

        # Add comparison if multiple crops
        if len(crop_list) > 1:
            market_data['comparison'] = self._compare_crop_prices(
                market_data['prices_by_crop']
            )

        return market_data

    def _get_single_crop_prices(self, crop_type: str) -> Dict:
        """Get market prices for a single crop"""

        if crop_type == 'maize':
            return {
                'crop': 'Maize',
                'regional_prices': {
                    'Kano': '₦32,000 - ₦37,000 per 100kg bag',
                    'Lagos': '₦35,000 - ₦40,000 per 100kg bag',
                    'Kaduna': '₦33,000 - ₦38,000 per 100kg bag',
                    'Sokoto': '₦31,000 - ₦36,000 per 100kg bag'
                },
                'price_range': '₦31,000 - ₦40,000 per 100kg',
                'average_price': '₦35,000 per 100kg',
                'trend': 'Stable with slight seasonal increase',
                'advice': [
                    'Prices are currently favorable for selling',
                    'Demand is steady approaching dry season',
                    'Quality premium: Disease-free maize fetches higher prices'
                ]
            }

        elif crop_type == 'rice':
            return {
                'crop': 'Rice',
                'regional_prices': {
                    'Kano': '₦42,000 - ₦47,000 per 100kg bag',
                    'Lagos': '₦45,000 - ₦50,000 per 100kg bag',
                    'Kaduna': '₦43,000 - ₦48,000 per 100kg bag',
                    'Sokoto': '₦41,000 - ₦46,000 per 100kg bag'
                },
                'price_range': '₦41,000 - ₦50,000 per 100kg',
                'average_price': '₦45,000 per 100kg',
                'trend': 'Strong and stable',
                'advice': [
                    'Rice prices remain strong',
                    'Good time to harvest and sell',
                    'Premium for clean, disease-free grain'
                ]
            }

        elif crop_type == 'sorghum':
            return {
                'crop': 'Sorghum',
                'regional_prices': {
                    'Kano': '₦28,000 - ₦33,000 per 100kg bag',
                    'Lagos': '₦30,000 - ₦35,000 per 100kg bag',
                    'Kaduna': '₦29,000 - ₦34,000 per 100kg bag',
                    'Sokoto': '₦27,000 - ₦32,000 per 100kg bag'
                },
                'price_range': '₦27,000 - ₦35,000 per 100kg',
                'average_price': '₦31,000 per 100kg',
                'trend': 'Stable',
                'advice': ['Steady demand for sorghum', 'Prices stable year-round']
            }

        elif crop_type == 'groundnut':
            return {
                'crop': 'Groundnut',
                'regional_prices': {
                    'Kano': '₦45,000 - ₦52,000 per 100kg bag',
                    'Lagos': '₦48,000 - ₦55,000 per 100kg bag',
                    'Kaduna': '₦46,000 - ₦53,000 per 100kg bag',
                    'Sokoto': '₦44,000 - ₦51,000 per 100kg bag'
                },
                'price_range': '₦44,000 - ₦55,000 per 100kg',
                'average_price': '₦49,000 per 100kg',
                'trend': 'Rising',
                'advice': ['High demand', 'Excellent prices currently', 'Good time to sell']
            }

        else:
            return {'error': f'Price data not available for {crop_type}'}

    def _compare_crop_prices(self, prices_by_crop: Dict) -> Dict:
        """Compare prices across multiple crops"""

        comparison = {
            'highest_average': None,
            'lowest_average': None,
            'best_for_selling': [],
            'summary': ''
        }

        # Extract average prices
        avg_prices = {}
        for crop, data in prices_by_crop.items():
            if 'average_price' in data:
                # Extract number from string like "₦35,000 per 100kg"
                price_str = data['average_price'].replace('₦', '').replace(',', '')
                price_str = price_str.split()[0]  # Get just the number
                avg_prices[crop] = int(price_str)

        if avg_prices:
            # Find highest and lowest
            highest = max(avg_prices.items(), key=lambda x: x[1])
            lowest = min(avg_prices.items(), key=lambda x: x[1])

            comparison['highest_average'] = {
                'crop': highest[0],
                'price': f"₦{highest[1]:,}"
            }
            comparison['lowest_average'] = {
                'crop': lowest[0],
                'price': f"₦{lowest[1]:,}"
            }

            # Identify best for selling (high prices + good trend)
            for crop, data in prices_by_crop.items():
                if data.get('trend', '').lower() in ['rising', 'strong', 'strong and stable']:
                    comparison['best_for_selling'].append(crop)

            comparison['summary'] = f"{highest[0]} has highest average price at ₦{highest[1]:,} per 100kg"

        return comparison

    def _generate_farming_advisory(self, weather_data: Dict,
                                   market_data: Dict,
                                   disease_info: Dict = None) -> List[str]:
        """Generate actionable farming advice"""

        advisories = []

        # Weather-based advice
        if weather_data.get('humidity_range', {}).get('max', 50) < 50:
            advisories.append("✓ Low humidity conditions good for harvest and storage")

        # Market-based advice
        if market_data.get('comparison', {}).get('best_for_selling'):
            best_crops = market_data['comparison']['best_for_selling']
            advisories.append(
                f"💰 Best market opportunities: {', '.join(best_crops)}"
            )

        # Disease + weather + market combined
        if disease_info:
            severity = disease_info.get('severity', '').lower()

            if 'moderate' in severity or 'severe' in severity:
                advisories.append("🚨 URGENT: Treat disease to preserve crop value")
                advisories.append("📅 Target harvest in 2-3 weeks after treatment")

        return advisories

    def format_context_report(self, context_data: Dict) -> str:
        """Format geographical context into readable report"""

        report = f"""
{'=' * 70}
🌍 GEO-CONTEXT INTELLIGENCE REPORT
{'=' * 70}

📍 LOCATION: {context_data['location'].upper()}

🌤️ WEATHER FORECAST ({context_data['weather'].get('forecast_period', '7 days')}):
   Temperature: {context_data['weather'].get('temperature', 'N/A')}
   Humidity: {context_data['weather'].get('humidity', 'N/A')}
   Rainfall: {context_data['weather'].get('rainfall', 'N/A')}
   Conditions: {context_data['weather'].get('conditions', 'N/A')}
"""

        if context_data['weather'].get('crop_specific_advice'):
            report += f"\n   🌾 Crop-Specific Weather Advice:\n"
            for advice in context_data['weather']['crop_specific_advice']:
                report += f"   {advice}\n"

        if context_data['weather'].get('disease_relevance'):
            report += f"\n   🔬 Disease-Weather Context:\n"
            for relevance in context_data['weather']['disease_relevance']:
                report += f"   {relevance}\n"

        # Market Intelligence Section
        market = context_data.get('market', {})

        if market.get('prices_by_crop'):
            report += f"\n💰 MARKET INTELLIGENCE:\n"
            report += f"   Crops Queried: {', '.join(market.get('crops_queried', []))}\n\n"

            for crop, data in market['prices_by_crop'].items():
                if data.get('error'):
                    report += f"   {crop.upper()}: {data['error']}\n\n"
                    continue

                report += f"   📊 {crop.upper()}:\n"
                report += f"      Price Range: {data.get('price_range', 'N/A')}\n"
                report += f"      Average: {data.get('average_price', 'N/A')}\n"
                report += f"      Trend: {data.get('trend', 'N/A')}\n"

                if data.get('regional_prices'):
                    report += f"      Regional Prices:\n"
                    for loc, price in list(data['regional_prices'].items())[:3]:
                        report += f"         • {loc}: {price}\n"

                if data.get('advice'):
                    report += f"      Market Advice:\n"
                    for advice in data['advice'][:2]:
                        report += f"         • {advice}\n"
                report += "\n"

            # Add comparison if multiple crops
            if market.get('comparison', {}).get('summary'):
                report += f"   📈 PRICE COMPARISON:\n"
                report += f"      {market['comparison']['summary']}\n"
                if market['comparison'].get('best_for_selling'):
                    report += f"      Best for selling now: {', '.join(market['comparison']['best_for_selling'])}\n"

        if context_data.get('farming_advisory'):
            report += f"\n📋 INTEGRATED FARMING ADVISORY:\n"
            for advisory in context_data['farming_advisory']:
                report += f"   {advisory}\n"

        report += f"\n{'=' * 70}\n"

        return report

    def get_statistics(self) -> Dict[str, any]:
        """Get agent statistics"""
        return {
            'agent_name': self.agent_name,
            'total_analyses': self.analyses_count,
            'supported_crops': self.supported_crops
        }

# Re-initialize Geo-Context Agent with new capabilities
geo_context = GeoContextAgent(quota_manager=quota_manager)

print("✅ Geo-Context Agent initialized (REVISED)")
print(f"   Flexible market queries for user-specified crops")
print(f"   Supported crops: {', '.join(geo_context.supported_crops)}")
print("=" * 70)

In [ ]:
# Cell 12: Flexible Coordinator Agent
# Handles multiple input scenarios and workflows

class CoordinatorAgent:
    """
    Main orchestrator that:
    1. Accepts flexible user inputs (image, location, market queries)
    2. Routes to appropriate specialist agents based on what user needs
    3. Combines all insights into comprehensive report
    4. Manages workflow and observability
    """

    def __init__(self, crop_pathologist, quality_assessor, geo_context, quota_manager):
        self.crop_pathologist = crop_pathologist
        self.quality_assessor = quality_assessor
        self.geo_context = geo_context
        self.quota_manager = quota_manager
        self.agent_name = "Coordinator"
        self.analyses_count = 0
        self.execution_logs = []

    def process_request(self,
                       image_path: str = None,
                       location: str = None,
                       crop_type: str = None,
                       market_crops: List[str] = None,
                       include_quality: bool = True,
                       include_weather: bool = True) -> Dict[str, any]:
        """
        Process farmer request with flexible inputs

        USER INPUTS (flexible combinations):

        SCENARIO 1 - Disease Diagnosis + Full Intelligence:
          - image_path: Path to crop photo (required)
          - location: Farmer's location (required for weather/market)
          - crop_type: Optional - 'maize' or 'rice' (auto-detect if not provided)
          - market_crops: Optional - additional crops to check prices for
          - include_quality: True to assess overall health
          - include_weather: True to get weather forecast

        SCENARIO 2 - Market Intelligence Only:
          - location: Farmer's location (required)
          - market_crops: List of crops to get prices for (required)
          - image_path: None

        SCENARIO 3 - Weather Forecast Only:
          - location: Farmer's location (required)
          - crop_type: Crop for weather-specific advice
          - image_path: None

        Returns:
            Complete analysis report based on requested services
        """

        print("=" * 70)
        print("🎯 AGRICONNECT COORDINATOR - PROCESSING REQUEST")
        print("=" * 70)

        start_time = time.time()

        # Determine workflow based on inputs
        has_image = image_path and Path(image_path).exists()
        has_location = location is not None
        wants_market = market_crops is not None and len(market_crops) > 0

        # Validate workflow
        if not has_image and not has_location:
            return {
                'error': 'Please provide either an image for diagnosis OR a location for market/weather intelligence'
            }

        # Log user inputs
        print(f"\n📥 USER INPUTS:")
        if has_image:
            print(f"   • Image: {Path(image_path).name}")
        if has_location:
            print(f"   • Location: {location}")
        if crop_type:
            print(f"   • Crop Type: {crop_type}")
        if wants_market:
            print(f"   • Market Query: {', '.join(market_crops)}")

        self._log(f"Request started - Image: {has_image}, Location: {has_location}, Market: {wants_market}")

        # Initialize result structure
        result = {
            'user_inputs': {
                'image_provided': has_image,
                'image_path': str(image_path) if has_image else None,
                'location': location,
                'crop_type': crop_type,
                'market_crops': market_crops
            },
            'agents_used': [],
            'execution_sequence': [],
            'workflow_type': ''
        }

        # WORKFLOW 1: Image-based disease diagnosis
        if has_image:
            result['workflow_type'] = 'Full Diagnosis with Intelligence'

            # Auto-detect crop type if not provided
            if not crop_type:
                crop_type = self._detect_crop_type(image_path)
                print(f"   • Detected Crop: {crop_type.upper()}")
                result['user_inputs']['crop_type'] = crop_type

            # STEP 1: Disease Diagnosis
            print(f"\n{'=' * 70}")
            print("STEP 1: DISEASE DIAGNOSIS")
            print("=" * 70)

            self._log("Routing to Crop Pathologist Agent")
            result['execution_sequence'].append("Crop Pathologist Agent")

            diagnosis = self.crop_pathologist.analyze_image(image_path, crop_type)
            result['diagnosis'] = diagnosis
            result['agents_used'].append('Crop Pathologist')

            if diagnosis.get('error'):
                print(f"   ❌ Diagnosis failed: {diagnosis['error']}")
                self._log(f"ERROR: Diagnosis failed - {diagnosis['error']}")
            else:
                print(f"   ✅ Diagnosis complete: {diagnosis['condition']}")
                self._log(f"Diagnosis: {diagnosis['condition']} (Confidence: {diagnosis['confidence']})")

            # STEP 2: Quality Assessment
            if include_quality:
                print(f"\n{'=' * 70}")
                print("STEP 2: QUALITY ASSESSMENT")
                print("=" * 70)

                self._log("Routing to Quality Assessor Agent")
                result['execution_sequence'].append("Quality Assessor Agent")

                assessment = self.quality_assessor.assess_health(
                    image_path, crop_type, diagnosis
                )
                result['assessment'] = assessment
                result['agents_used'].append('Quality Assessor')

                if assessment.get('error'):
                    print(f"   ❌ Assessment failed: {assessment['error']}")
                    self._log(f"ERROR: Assessment failed - {assessment['error']}")
                else:
                    print(f"   ✅ Assessment complete: {assessment['health_score']}/100")
                    self._log(f"Health Score: {assessment['health_score']}/100")

        # WORKFLOW 2 or 3: Geo-Context (Weather and/or Market)
        if has_location and (include_weather or wants_market):
            print(f"\n{'=' * 70}")
            print("STEP 3: GEO-CONTEXT INTELLIGENCE")
            print("=" * 70)

            if not has_image:
                result['workflow_type'] = 'Market/Weather Intelligence Only'

            # Determine which crops to get market data for
            final_market_crops = []

            # If user specified market crops, use those
            if wants_market:
                final_market_crops = market_crops
            # If doing image diagnosis, include that crop automatically
            elif has_image and crop_type:
                final_market_crops = [crop_type]

            self._log(f"Routing to Geo-Context Agent for {location}")
            result['execution_sequence'].append("Geo-Context Agent")

            # Get context with flexible parameters
            context = self.geo_context.get_context(
                location=location,
                weather_crop=crop_type if has_image else None,
                market_crops=final_market_crops if final_market_crops else None,
                disease_info=result.get('diagnosis') if has_image else None
            )

            result['context'] = context
            result['agents_used'].append('Geo-Context')

            if context.get('error'):
                print(f"   ❌ Context gathering failed: {context['error']}")
                self._log(f"ERROR: Context failed - {context['error']}")
            else:
                print(f"   ✅ Context gathered for {location}")
                if final_market_crops:
                    print(f"   ✅ Market prices for: {', '.join(final_market_crops)}")
                self._log(f"Context: Weather & Market data for {location}")

        # STEP 4: Generate Combined Insights
        if has_image or (has_location and wants_market):
            print(f"\n{'=' * 70}")
            print("STEP 4: GENERATING COMBINED INSIGHTS")
            print("=" * 70)

            combined = self._generate_combined_insights(result)
            result['combined_insights'] = combined

        # Calculate total execution time
        total_time = time.time() - start_time
        result['metadata'] = {
            'agent': self.agent_name,
            'total_execution_time': total_time,
            'agents_used': result['agents_used'],
            'execution_sequence': result['execution_sequence'],
            'workflow_type': result['workflow_type'],
            'timestamp': datetime.now().isoformat()
        }

        self.analyses_count += 1
        self._log(f"Request complete in {total_time:.2f}s")

        print(f"\n✅ PROCESSING COMPLETE")
        print(f"   Workflow: {result['workflow_type']}")
        print(f"   Time: {total_time:.2f}s")
        print(f"   Agents used: {', '.join(result['agents_used'])}")
        print("=" * 70)

        return result

    def _detect_crop_type(self, image_path: str) -> str:
        """Auto-detect crop type from image path"""
        path_str = str(image_path).lower()

        if 'maize' in path_str or 'corn' in path_str:
            return 'maize'
        elif 'rice' in path_str or 'paddy' in path_str:
            return 'rice'

        return 'maize'  # Default

    def _generate_combined_insights(self, result: Dict) -> Dict:
        """Generate actionable insights from all agent outputs"""

        insights = {
            'priority_actions': [],
            'timeline': [],
            'economic_impact': {},
            'risk_level': 'Unknown'
        }

        # Extract key data
        diagnosis = result.get('diagnosis', {})
        assessment = result.get('assessment', {})
        context = result.get('context', {})

        # If we have diagnosis, generate disease-focused insights
        if diagnosis and not diagnosis.get('error'):
            condition = diagnosis.get('condition', 'Unknown')
            severity = diagnosis.get('severity', 'Unknown')
            health_score = assessment.get('health_score', 0) if assessment else 0

            # Determine risk level
            if 'severe' in severity.lower() or health_score < 40:
                insights['risk_level'] = 'HIGH - URGENT ACTION NEEDED'
                insights['priority_actions'].append(
                    "🚨 URGENT: Begin treatment immediately (within 24-48 hours)"
                )
            elif 'moderate' in severity.lower() or 40 <= health_score < 60:
                insights['risk_level'] = 'MODERATE - ACTION RECOMMENDED'
                insights['priority_actions'].append(
                    "⚠️ IMPORTANT: Begin treatment within 3-5 days"
                )
            else:
                insights['risk_level'] = 'LOW - MONITOR CLOSELY'
                insights['priority_actions'].append(
                    "✓ Continue monitoring, preventive measures recommended"
                )

            # Add treatment-specific actions
            if diagnosis.get('treatment'):
                insights['priority_actions'].append(
                    f"Apply: {diagnosis['treatment'][0]}"
                )

        # If we have market data, generate market-focused insights
        if context and context.get('market', {}).get('prices_by_crop'):
            market = context['market']

            # Add market timing advice
            if market.get('comparison', {}).get('best_for_selling'):
                best_crops = market['comparison']['best_for_selling']
                insights['priority_actions'].append(
                    f"💰 Best selling opportunity: {', '.join(best_crops)}"
                )

            # Economic analysis
            if market.get('comparison', {}).get('highest_average'):
                highest = market['comparison']['highest_average']
                insights['economic_impact']['best_crop'] = highest['crop']
                insights['economic_impact']['best_price'] = highest['price']

        # Add timeline from farming advisory
        if context and context.get('farming_advisory'):
            insights['timeline'] = context['farming_advisory']

        return insights

    def _log(self, message: str):
        """Log execution events for observability"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'agent': self.agent_name,
            'message': message
        }
        self.execution_logs.append(log_entry)

    def format_complete_report(self, result: Dict) -> str:
        """Format the complete analysis into a farmer-friendly report"""

        report = f"""
╔════════════════════════════════════════════════════════════════════╗
║                 🌾 AGRICONNECT INTELLIGENCE REPORT 🌾              ║
║              Agricultural Intelligence Multi-Agent System          ║
╚════════════════════════════════════════════════════════════════════╝

📅 REPORT DATE: {datetime.now().strftime('%B %d, %Y at %H:%M')}
📍 LOCATION: {result['user_inputs']['location'].upper() if result['user_inputs']['location'] else 'N/A'}
🔄 WORKFLOW: {result.get('workflow_type', 'Unknown')}

"""

        # Disease Diagnosis Section (if available)
        if 'diagnosis' in result and not result['diagnosis'].get('error'):
            report += f"🌾 CROP: {result['user_inputs']['crop_type'].upper()}\n\n"
            report += self.crop_pathologist.format_diagnosis_report(result['diagnosis'])

        # Quality Assessment Section (if available)
        if 'assessment' in result and not result['assessment'].get('error'):
            report += self.quality_assessor.format_assessment_report(result['assessment'])

        # Geo Context Section (if available)
        if 'context' in result and not result['context'].get('error'):
            report += self.geo_context.format_context_report(result['context'])

        # Combined Insights Section
        insights = result.get('combined_insights', {})
        if insights and insights.get('priority_actions'):
            report += f"""
{'=' * 70}
🎯 INTEGRATED RECOMMENDATIONS & ACTION PLAN
{'=' * 70}
"""

            if insights.get('risk_level') and insights['risk_level'] != 'Unknown':
                report += f"\n⚠️ RISK LEVEL: {insights['risk_level']}\n"

            report += f"\n🔴 PRIORITY ACTIONS:\n"
            for action in insights.get('priority_actions', []):
                report += f"   {action}\n"

            if insights.get('economic_impact'):
                report += f"\n💰 ECONOMIC INSIGHTS:\n"
                impact = insights['economic_impact']
                if impact.get('best_crop'):
                    report += f"   • Highest value crop: {impact['best_crop']} at {impact.get('best_price', 'N/A')}\n"
                if impact.get('potential_yield_loss'):
                    report += f"   • Potential Yield Loss: {impact['potential_yield_loss']}\n"

        # Execution Metadata
        metadata = result.get('metadata', {})
        report += f"""
{'=' * 70}
📊 ANALYSIS METADATA
{'=' * 70}
   Execution Time: {metadata.get('total_execution_time', 0):.2f} seconds
   Agents Used: {', '.join(metadata.get('agents_used', []))}
   Workflow: {metadata.get('workflow_type', 'Unknown')}

{'=' * 70}
✅ AgriConnect - Empowering Nigerian Farmers with AI
   For questions or support, consult your local agricultural extension officer
{'=' * 70}
"""

        return report

    def get_statistics(self) -> Dict:
        """Get coordinator statistics"""
        return {
            'agent_name': self.agent_name,
            'total_analyses': self.analyses_count,
            'execution_logs': len(self.execution_logs)
        }

    def print_execution_log(self, last_n: int = 10):
        """Print execution log for observability"""
        print(f"\n📋 EXECUTION LOG (Last {last_n} entries)")
        print("=" * 70)
        for log in self.execution_logs[-last_n:]:
            print(f"[{log['timestamp']}] {log['message']}")
        print("=" * 70)

# Re-initialize Coordinator Agent with updated capabilities
coordinator = CoordinatorAgent(
    crop_pathologist=crop_pathologist,
    quality_assessor=quality_assessor,
    geo_context=geo_context,
    quota_manager=quota_manager
)

print("✅ Coordinator Agent initialized (REVISED)")
print(f"   Supports flexible workflows:")
print(f"   • Full diagnosis + intelligence")
print(f"   • Market intelligence only")
print(f"   • Weather forecast only")
print("=" * 70)

In [ ]:
# Cell 14: Translation Agent - Makes reports accessible in Hausa

class TranslationAgent:
    """
    Specialist agent for translating complete reports to Hausa
    Uses Gemini API to provide culturally-appropriate translations
    """

    def __init__(self, gemini_client, model_name: str, quota_manager):
        self.client = gemini_client
        self.model_name = model_name
        self.quota_manager = quota_manager
        self.agent_name = "Translation"
        self.translations_count = 0

    def translate_to_hausa(self, english_report: str) -> Dict[str, any]:
        """
        Translate complete English report to Hausa

        Args:
            english_report: Complete report in English from all agents

        Returns:
            Dictionary with Hausa translation and metadata
        """

        print(f"\n🌍 [{self.agent_name}] Starting translation to Hausa...")
        print(f"   Report length: {len(english_report)} characters")

        start_time = time.time()

        try:
            # Create comprehensive translation prompt
            prompt = f"""You are an expert agricultural translator specializing in Nigerian farming terminology.

Your task is to translate this COMPLETE agricultural intelligence report from English to Hausa for farmers in Northern Nigeria.

TRANSLATION GUIDELINES:

1. **Language Style**:
   - Use clear, simple Hausa that farmers understand
   - Be respectful and professional
   - Use active voice where possible

2. **Agricultural Terminology**:
   Key Hausa terms to use consistently:
   - Disease/Illness: Cuta
   - Leaf/Leaves: Ganye (singular), Ganyen (plural/possessive)
   - Treatment/Medicine: Magani, Jiyya
   - Prevention/Protection: Rigakafi, Karewa
   - Symptoms/Signs: Alamomi, Shaida
   - Farmer: Manomi (singular), Manoma (plural)
   - Crop/Farm: Amfanin gona, Gona
   - Harvest: Girbi
   - Plant/Planting: Shuka
   - Spray: Fesa
   - Apply: Shafa, Yi amfani

   Crops:
   - Maize: Masara
   - Rice: Shinkafa
   - Sorghum: Dawa
   - Groundnut: Gyada

   Market terms:
   - Price: Farashi
   - Market: Kasuwa
   - Sell: Sayar
   - Buy: Saya

   Weather terms:
   - Weather: Yanayi
   - Rain: Ruwan sama
   - Temperature: Zafin iska
   - Humidity: Danshi
   - Wind: Iska

3. **Special Instructions**:
   - Keep disease names in English but add Hausa context: e.g., "Common Rust (Tsatsa)"
   - Preserve ALL formatting (headers, bullet points, sections)
   - Keep emojis and symbols (🌾, 📊, 💰, etc.)
   - Translate section headers clearly
   - Keep currency symbols: ₦
   - Maintain numbered lists and structure
   - Keep technical/scientific names in English with Hausa explanation

4. **Cultural Adaptation**:
   - Be encouraging and supportive in tone
   - Emphasize actionable steps clearly
   - Respect local farming practices

IMPORTANT: Provide ONLY the Hausa translation. Do not add explanations, notes, or commentary.

ENGLISH REPORT TO TRANSLATE:
{english_report}

HAUSA TRANSLATION:"""

            # Wait for quota availability
            self.quota_manager.wait_if_needed(self.agent_name)

            # Make API call
            print(f"   🤖 Calling Gemini for translation...")
            print(f"   ⏳ This may take 15-20 seconds for long reports...")

            response = self.client.models.generate_content(
                model=self.model_name,
                contents=prompt
            )

            # Record API usage
            self.quota_manager.record_request(self.agent_name)
            self.translations_count += 1

            elapsed_time = time.time() - start_time

            hausa_text = response.text.strip()

            # Clean up any markdown artifacts if present
            hausa_text = hausa_text.replace('```hausa', '').replace('```', '').strip()

            print(f"   ✓ Translation complete in {elapsed_time:.2f}s")
            print(f"   ✓ Hausa text length: {len(hausa_text)} characters")

            translation_data = {
                'hausa_text': hausa_text,
                'english_length': len(english_report),
                'hausa_length': len(hausa_text),
                'metadata': {
                    'agent': self.agent_name,
                    'model': self.model_name,
                    'translation_time': elapsed_time,
                    'timestamp': datetime.now().isoformat()
                }
            }

            print(f"   ✅ Translation successful!")

            return translation_data

        except Exception as e:
            error_time = time.time() - start_time
            print(f"   ❌ Translation failed after {error_time:.2f}s: {str(e)[:100]}")

            return {
                'hausa_text': f"[Translation Error: {str(e)}]\n\n{english_report}",
                'error': str(e),
                'metadata': {
                    'agent': self.agent_name,
                    'error_time': error_time
                }
            }

    def create_bilingual_report(self, english_report: str) -> str:
        """
        Create bilingual report with both English and Hausa

        Args:
            english_report: Complete English report

        Returns:
            Formatted bilingual report
        """

        print(f"\n🌍 [{self.agent_name}] Creating bilingual report...")

        # Translate to Hausa
        translation_data = self.translate_to_hausa(english_report)

        # Combine into bilingual format
        bilingual_report = f"""
{'=' * 70}
🌾 AGRICONNECT - BILINGUAL REPORT / RAHOTON BIYU
{'=' * 70}

📱 ENGLISH VERSION / TURANCI
{'=' * 70}

{english_report}

{'=' * 70}
{'=' * 70}
🌍 HAUSA TRANSLATION / FASSARAR HAUSA
{'=' * 70}
{'=' * 70}

{translation_data['hausa_text']}

{'=' * 70}
✅ AgriConnect - Helping Farmers / Taimakawa Manoma
   English & Hausa | Turanci da Hausa
{'=' * 70}
"""

        return bilingual_report

    def get_statistics(self) -> Dict[str, any]:
        """Get agent statistics"""
        return {
            'agent_name': self.agent_name,
            'total_translations': self.translations_count,
            'api_calls_made': self.quota_manager.by_agent.get(self.agent_name, 0)
        }

# Initialize Translation Agent
translation_agent = TranslationAgent(
    gemini_client=gemini_client,
    model_name=GEMINI_MODEL,
    quota_manager=quota_manager
)

print("✅ Translation Agent initialized")
print(f"   Using model: {GEMINI_MODEL}")
print(f"   Translates complete reports to Hausa")
print(f"   Provides bilingual output for accessibility")
print("=" * 70)

In [ ]:
# Cell 15: Add translation capability to Coordinator

# Add translation agent to coordinator
coordinator.translation_agent = translation_agent

# Add method to coordinator for bilingual reports
def generate_bilingual_report(self, result: Dict, language: str = 'both') -> str:
    """
    Generate report in requested language(s)

    Args:
        result: Analysis result from process_request
        language: 'english', 'hausa', or 'both'

    Returns:
        Report in requested language(s)
    """

    # Generate English report
    english_report = self.format_complete_report(result)

    if language == 'english':
        return english_report

    elif language == 'hausa':
        # Translate to Hausa only
        translation_data = self.translation_agent.translate_to_hausa(english_report)
        return translation_data['hausa_text']

    else:  # 'both'
        # Create bilingual report
        return self.translation_agent.create_bilingual_report(english_report)

# Add method to coordinator class
CoordinatorAgent.generate_bilingual_report = generate_bilingual_report

print("✅ Coordinator updated with translation capability")
print("   Can now generate reports in:")
print("   • English only")
print("   • Hausa only")
print("   • Bilingual (English + Hausa)")
print("=" * 70)

In [ ]:
# Cell 17: Memory Bank - Stores farmer profiles and diagnosis history

import json
from datetime import datetime
from typing import List, Optional

class MemoryBank:
    """
    Memory system for AgriConnect
    - Stores farmer profiles (location, crops, preferences)
    - Tracks diagnosis history
    - Enables personalized recommendations
    """

    def __init__(self):
        self.farmers = {}  # farmer_id -> profile
        self.diagnoses_history = []  # All diagnoses
        self.agent_name = "Memory Bank"

    def create_farmer_profile(self, farmer_id: str, location: str,
                             name: str = None, language: str = 'both') -> Dict:
        """
        Create or update farmer profile

        Args:
            farmer_id: Unique farmer identifier (e.g., phone number)
            location: Farmer's location
            name: Farmer's name (optional)
            language: Preferred language ('english', 'hausa', 'both')
        """

        if farmer_id not in self.farmers:
            profile = {
                'farmer_id': farmer_id,
                'name': name,
                'location': location,
                'language_preference': language,
                'crops_diagnosed': [],
                'total_diagnoses': 0,
                'last_contact': datetime.now().isoformat(),
                'created_at': datetime.now().isoformat()
            }
            self.farmers[farmer_id] = profile
            print(f"✅ New farmer profile created: {farmer_id}")
        else:
            # Update existing profile
            self.farmers[farmer_id]['location'] = location
            self.farmers[farmer_id]['last_contact'] = datetime.now().isoformat()
            if name:
                self.farmers[farmer_id]['name'] = name
            print(f"✅ Farmer profile updated: {farmer_id}")

        return self.farmers[farmer_id]

    def save_diagnosis(self, farmer_id: str, diagnosis_result: Dict):
        """
        Save a diagnosis to farmer's history

        Args:
            farmer_id: Farmer identifier
            diagnosis_result: Complete result from coordinator
        """

        # Extract key information
        diagnosis_record = {
            'record_id': len(self.diagnoses_history) + 1,
            'farmer_id': farmer_id,
            'timestamp': datetime.now().isoformat(),
            'location': diagnosis_result.get('user_inputs', {}).get('location'),
            'crop_type': diagnosis_result.get('user_inputs', {}).get('crop_type'),
            'diagnosis': diagnosis_result.get('diagnosis', {}).get('condition'),
            'confidence': diagnosis_result.get('diagnosis', {}).get('confidence'),
            'severity': diagnosis_result.get('diagnosis', {}).get('severity'),
            'health_score': diagnosis_result.get('assessment', {}).get('health_score'),
            'risk_level': diagnosis_result.get('combined_insights', {}).get('risk_level'),
            'workflow_type': diagnosis_result.get('workflow_type')
        }

        # Add to global history
        self.diagnoses_history.append(diagnosis_record)

        # Update farmer profile
        if farmer_id in self.farmers:
            self.farmers[farmer_id]['total_diagnoses'] += 1

            crop = diagnosis_record['crop_type']
            if crop and crop not in self.farmers[farmer_id]['crops_diagnosed']:
                self.farmers[farmer_id]['crops_diagnosed'].append(crop)

        print(f"💾 Diagnosis saved: Record #{diagnosis_record['record_id']} for {farmer_id}")
        return diagnosis_record

    def get_farmer_history(self, farmer_id: str) -> List[Dict]:
        """Get all diagnoses for a specific farmer"""
        return [d for d in self.diagnoses_history if d['farmer_id'] == farmer_id]

    def get_farmer_profile(self, farmer_id: str) -> Optional[Dict]:
        """Get farmer profile"""
        return self.farmers.get(farmer_id)

    def get_crop_statistics(self, crop_type: str) -> Dict:
        """Get statistics for a specific crop across all farmers"""
        crop_diagnoses = [d for d in self.diagnoses_history
                         if d.get('crop_type') == crop_type]

        if not crop_diagnoses:
            return {'crop': crop_type, 'total_diagnoses': 0}

        # Count diseases
        diseases = {}
        for d in crop_diagnoses:
            disease = d.get('diagnosis', 'Unknown')
            diseases[disease] = diseases.get(disease, 0) + 1

        return {
            'crop': crop_type,
            'total_diagnoses': len(crop_diagnoses),
            'most_common_disease': max(diseases.items(), key=lambda x: x[1])[0],
            'disease_distribution': diseases,
            'unique_farmers': len(set(d['farmer_id'] for d in crop_diagnoses))
        }

    def generate_farmer_summary(self, farmer_id: str) -> str:
        """Generate a summary report for a farmer"""

        profile = self.get_farmer_profile(farmer_id)
        history = self.get_farmer_history(farmer_id)

        if not profile:
            return f"❌ No profile found for farmer: {farmer_id}"

        summary = f"""
{'=' * 70}
👨‍🌾 FARMER PROFILE SUMMARY
{'=' * 70}

📋 FARMER INFORMATION:
   ID: {profile['farmer_id']}
   Name: {profile.get('name', 'N/A')}
   Location: {profile['location']}
   Language Preference: {profile['language_preference']}
   Member Since: {profile['created_at'][:10]}
   Last Contact: {profile['last_contact'][:10]}

📊 USAGE STATISTICS:
   Total Diagnoses: {profile['total_diagnoses']}
   Crops Diagnosed: {', '.join(profile['crops_diagnosed']) if profile['crops_diagnosed'] else 'None'}

📜 RECENT DIAGNOSIS HISTORY:
"""

        if history:
            for record in history[-5:]:  # Last 5 diagnoses
                summary += f"""
   {record['timestamp'][:10]} - {record['crop_type'].upper() if record['crop_type'] else 'N/A'}
      Disease: {record['diagnosis']}
      Severity: {record['severity']} | Health: {record['health_score']}/100
"""
        else:
            summary += "   No diagnosis history\n"

        summary += f"\n{'=' * 70}\n"

        return summary

    def get_statistics(self) -> Dict:
        """Get overall memory bank statistics"""
        return {
            'total_farmers': len(self.farmers),
            'total_diagnoses': len(self.diagnoses_history),
            'unique_crops': len(set(d.get('crop_type') for d in self.diagnoses_history if d.get('crop_type'))),
            'active_farmers': len([f for f in self.farmers.values()
                                  if f['total_diagnoses'] > 0])
        }

# Initialize Memory Bank
memory_bank = MemoryBank()

print("✅ Memory Bank initialized")
print("   Stores farmer profiles and diagnosis history")
print("   Enables personalized recommendations")
print("=" * 70)

In [ ]:
# Cell 22: Demo with FULL Hausa Explanation

def agriconnect_demo_hausa(user_input, input_type: str, location: str = "Kano"):
    """
    Demo interface with COMPLETE Hausa output
    """

    print("=" * 70)
    print("🌾 AGRICONNECT DEMO")
    print("=" * 70)

    if input_type == 'image':
        # IMAGE → Full Hausa diagnosis
        print("\n📸 IMAGE UPLOAD DETECTED")
        print(f"Location: {location}")
        print("-" * 70)

        # Detect crop type
        crop_type = coordinator._detect_crop_type(user_input)

        # Run diagnosis
        result = coordinator.process_request(
            image_path=user_input,
            location=location,
            crop_type=crop_type,
            include_quality=False,
            include_weather=False
        )

        # Get diagnosis info
        diagnosis = result.get('diagnosis', {})

        # Create FULL HAUSA response using Translation Agent
        english_summary = f"""
Disease Diagnosis Report

Condition: {diagnosis.get('condition', 'Unknown')}
Confidence: {diagnosis.get('confidence', 'Unknown')}
Severity: {diagnosis.get('severity', 'Unknown')}

Treatment:
{chr(10).join('• ' + t for t in diagnosis.get('treatment', []))}

Prevention:
{chr(10).join('• ' + p for p in diagnosis.get('prevention', []))}

Special Notes:
{diagnosis.get('special_notes', '')}
"""

        print(f"\n🔄 Translating to Hausa...")

        # Translate to Hausa
        translation_result = translation_agent.translate_to_hausa(english_summary)
        hausa_text = translation_result.get('hausa_text', '')

        # Display in Hausa
        print(f"\n{'=' * 70}")
        print("🔬 RAHOTON GANO CUTA (Disease Diagnosis)")
        print("=" * 70)
        print(hausa_text)
        print("=" * 70)

        return result

    elif input_type == 'crop_price':
        # CROP PRICE → Hausa
        print("\n💰 MARKET PRICE QUERY")
        print(f"Crop: {user_input}")
        print(f"Location: {location}")
        print("-" * 70)

        # Get market prices
        context = geo_context.get_context(
            location=location,
            market_crops=[user_input.lower()]
        )

        market = context.get('market', {})
        crop_data = market.get('prices_by_crop', {}).get(user_input.lower(), {})

        # Create English summary
        english_summary = f"""
Market Price Report for {user_input.upper()}

Price Range: {crop_data.get('price_range', 'N/A')}
Average Price: {crop_data.get('average_price', 'N/A')}
Market Trend: {crop_data.get('trend', 'N/A')}

Regional Prices:
{chr(10).join(f"• {loc}: {price}" for loc, price in crop_data.get('regional_prices', {}).items())}

Market Advice:
{chr(10).join('• ' + a for a in crop_data.get('advice', []))}
"""

        print(f"\n🔄 Translating to Hausa...")

        # Translate to Hausa
        translation_result = translation_agent.translate_to_hausa(english_summary)
        hausa_text = translation_result.get('hausa_text', '')

        # Display in Hausa
        print(f"\n{'=' * 70}")
        print(f"💰 RAHOTON FARASHIN KASUWA - {user_input.upper()}")
        print("=" * 70)
        print(hausa_text)
        print("=" * 70)

        return context

    elif input_type == 'weather':
        # WEATHER → Hausa
        print("\n🌤️ WEATHER FORECAST QUERY")
        print(f"Location: {user_input}")
        print("-" * 70)

        # Get weather
        context = geo_context.get_context(
            location=user_input,
            market_crops=None
        )

        weather = context.get('weather', {})

        # Create English summary
        english_summary = f"""
Weather Forecast for {user_input.upper()}

Temperature: {weather.get('temperature', 'N/A')}
Humidity: {weather.get('humidity', 'N/A')}
Rainfall: {weather.get('rainfall', 'N/A')}
Conditions: {weather.get('conditions', 'N/A')}
Wind: {weather.get('wind', 'N/A')}

Farming Advisory:
{chr(10).join('• ' + a for a in weather.get('crop_specific_advice', []))}
"""

        print(f"\n🔄 Translating to Hausa...")

        # Translate to Hausa
        translation_result = translation_agent.translate_to_hausa(english_summary)
        hausa_text = translation_result.get('hausa_text', '')

        # Display in Hausa
        print(f"\n{'=' * 70}")
        print(f"🌤️ HASASHEN YANAYI - {user_input.upper()}")
        print("=" * 70)
        print(hausa_text)
        print("=" * 70)

        return context

    else:
        print(f"❌ Invalid input_type: {input_type}")
        return None

print("✅ Full Hausa Demo Interface Created")
print("   All outputs translated to Hausa using Gemini")
print("=" * 70)

In [ ]:
# Cell 22B: Setup for Interactive Demos
# Import libraries needed for file upload and display

from google.colab import files
from IPython.display import display, Image as IPImage

print("✅ Interactive demo libraries loaded")
print("   Ready for image uploads and display")

In [ ]:
# Cell 23: DEMO 1 - Upload Image, Get FULL Hausa Diagnosis

print("=" * 70)
print("📸 DEMO 1: IMAGE UPLOAD - COMPLETE HAUSA DIAGNOSIS")
print("=" * 70)

# Step 1: Get location
print("\n📍 STEP 1: Enter your location")
location = input("Location (e.g., Kano, Kaduna, Sokoto): ").strip()
if not location:
    location = "Kano"
    print(f"Using default: {location}")

# Step 2: Upload image
print("\n📤 STEP 2: Upload crop leaf image")
print("Click 'Choose Files' button below to upload your image...")

uploaded = files.upload()

if uploaded:
    # Get filename
    filename = list(uploaded.keys())[0]

    print(f"\n✅ Image received: {filename}")

    # Show the image
    print("\n📷 Your uploaded image:")
    display(IPImage(filename, width=500))

    # Step 3: Process with FULL HAUSA output
    print("\n🔄 STEP 3: Analyzing and translating to Hausa...")
    print("-" * 70)

    result = agriconnect_demo_hausa(
        user_input=filename,
        input_type='image',
        location=location
    )

    print("\n" + "=" * 70)
    print("✅ COMPLETE HAUSA DIAGNOSIS DELIVERED!")
    print("📱 Farmer can now understand everything in their language")
    print("=" * 70)

    # Show quota
    quota_manager.print_stats()

else:
    print("❌ No file uploaded. Please try again.")

In [ ]:
# Cell 24: DEMO 2 - Market Prices in FULL Hausa

print("=" * 70)
print("💰 DEMO 2: MARKET PRICES - COMPLETE HAUSA")
print("=" * 70)

# Step 1: Get location
print("\n📍 STEP 1: Enter your location")
location = input("Location (e.g., Kano, Kaduna, Lagos): ").strip()
if not location:
    location = "Kano"
    print(f"Using default: {location}")

# Step 2: Get crop name
print("\n🌾 STEP 2: Which crop prices do you want?")
print("Available crops: maize, rice, sorghum, groundnut")
crop = input("Enter crop name: ").strip().lower()

if not crop:
    crop = "maize"
    print(f"Using default: {crop}")

# Step 3: Get prices in Hausa
print(f"\n🔄 STEP 3: Getting prices and translating to Hausa...")
print("-" * 70)

agriconnect_demo_hausa(
    user_input=crop,
    input_type='crop_price',
    location=location
)

print("\n" + "=" * 70)
print("✅ COMPLETE HAUSA MARKET REPORT DELIVERED!")
print("📱 Farmer can read everything in Hausa")
print("=" * 70)

quota_manager.print_stats()

In [ ]:
# Cell 25: DEMO 3 - Weather Forecast in FULL Hausa

print("=" * 70)
print("🌤️ DEMO 3: WEATHER FORECAST - COMPLETE HAUSA")
print("=" * 70)

# Step 1: Get location
print("\n📍 STEP 1: Enter location for weather forecast")
location = input("Location (e.g., Kano, Sokoto, Kaduna): ").strip()

if not location:
    location = "Kano"
    print(f"Using default: {location}")

# Step 2: Get weather in Hausa
print(f"\n🔄 STEP 2: Getting forecast and translating to Hausa...")
print("-" * 70)

agriconnect_demo_hausa(
    user_input=location,
    input_type='weather'
)

print("\n" + "=" * 70)
print("✅ COMPLETE HAUSA WEATHER FORECAST DELIVERED!")
print("📱 Farmer understands weather advice in Hausa")
print("=" * 70)

quota_manager.print_stats()

In [ ]:
# Cell 26: QUICK DEMO - All in Hausa

print("⚡ QUICK DEMO - ALL OUTPUTS IN HAUSA")
print("=" * 70)
print("Perfect for video demonstration!\n")

# Demo 1: Maize Disease → Full Hausa
print("\n" + "=" * 70)
print("SCENARIO 1: Maize Disease - Complete Hausa Diagnosis")
print("=" * 70)

maize_blight = list((MAIZE_PATH / 'Blight').glob('*.jpg'))[0]
print(f"📷 Using image: {maize_blight.name}")
display(IPImage(str(maize_blight), width=400))

agriconnect_demo_hausa(
    user_input=str(maize_blight),
    input_type='image',
    location='Kano'
)

# Demo 2: Market Prices → Full Hausa
print("\n\n" + "=" * 70)
print("SCENARIO 2: Rice Prices - Complete Hausa Report")
print("=" * 70)

agriconnect_demo_hausa(
    user_input='rice',
    input_type='crop_price',
    location='Kaduna'
)

# Demo 3: Weather → Full Hausa
print("\n\n" + "=" * 70)
print("SCENARIO 3: Weather - Complete Hausa Forecast")
print("=" * 70)

agriconnect_demo_hausa(
    user_input='Sokoto',
    input_type='weather'
)

print("\n\n" + "=" * 70)
print("✅ ALL DEMOS COMPLETE - EVERYTHING IN HAUSA!")
print("🌾 Farmers can understand EVERYTHING in their language")
quota_manager.print_stats()
print("=" * 70)